<a href="https://colab.research.google.com/github/loject/temp/blob/master/Copy_of_Untitled5%20for%20up/down%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [4]:
import numpy as np
import re
import glob
import os
import time, datetime
import tensorflow
import pandas

from tensorflow.keras.layers import Dense, SimpleRNN, Input, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow
print("Tensorflow version " + tensorflow.__version__)

BASE_DIR = '/content/drive/MyDrive/CNNv2'
TRAIN_FILE = '/content/drive/MyDrive/CNNv2/formated.csv'

Tensorflow version 2.9.2


In [5]:
def find_in_str(st, sub_str, pos):
  last = 0
  for i in range(pos):
    last = st.find(sub_str, last + 1)
  return last

list_files = [i for i in glob.glob(BASE_DIR + "/*.csv") if i != TRAIN_FILE] 
open(TRAIN_FILE, 'w').close()
for i in list_files[:3]:
  print(os.path.abspath(i))
  with open(i, 'r', encoding='utf-8') as f:
      text = f.read()
      text = text.split('\n')
      for j in range(len(text)):
          elem = text[j]
          if len(elem) < 1: continue
          tm = datetime.datetime.strptime(elem.split(';')[3], '%H:%M:%S').time()
          tmp = datetime.datetime.strptime(elem.split(';')[2], '%d/%m/%y') + datetime.timedelta(hours=tm.hour, minutes=tm.minute, seconds=tm.second)
          text[j] = str(tmp.timestamp()) + elem[find_in_str(elem, ';', 4):]
      
      with open(TRAIN_FILE, 'a') as f:
          for i in text:
              f.write("%s\n" % i)

/content/drive/MyDrive/CNNv2/US1.BNGO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TBIO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TRIT_200120_220925.csv


In [6]:
# import os
# import csv
# import time, datetime

# def process_path(file_path):
#   with open(file_path) as f:
#     reader = csv.reader(f, delimiter=';')
#     data = [np.array(i) for i in reader if len(i) != 0]
#   inp_chars = 60
#   n = len(data) - inp_chars
#   X = np.array([data[i:i + inp_chars] for i in range(n) if len(data[i]) != 0])
#   Y = [i[4] for i in data[inp_chars:]]
#   return X, Y

# def crutch_wrap(file_path):
#   return tensorflow.numpy_function(process_path, [file_path], (tensorflow.string, tensorflow.float32))

# dataset = tensorflow.data.Dataset.list_files(TRAIN_FILE).map(crutch_wrap)

In [ ]:
import csv

with open(TRAIN_FILE) as f:
  reader = csv.reader(f, delimiter=';', quoting=csv.QUOTE_NONNUMERIC)
  data = [i for i in reader if len(i) != 0]

inp_chars = 60
n = len(data) - inp_chars
X = [data[i:i + inp_chars] for i in range(n) if len(data[i]) != 0]
Y = [i[4] for i in data[inp_chars:]]


model = Sequential()
model.add(Input((inp_chars, 6)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(LSTM(64))  # рекуррентный слой на 500 нейронов
model.add(Dense(12))
model.add(Dense(1))
model.summary()

model.compile(loss='mse', metrics=['MeanAbsoluteError'], optimizer=Adam(0.1))

history = model.fit(X, Y, batch_size=32, epochs=100)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                18176     
                                                                 
 dense (Dense)               (None, 12)                780       
                                                                 
 dense_1 (Dense)             (None, 1)                 13        
                                                                 
Total params: 18,969
Trainable params: 18,969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1976/2951 [===================>..........] - ETA: 28s - loss: 71.5409 - mean_absolute_error: 6.6409

In [ ]:
# model.save('/content/drive/MyDrive/CNNv2/some_shit_save')

In [ ]:
model.predict((X[0], X[0]))

1/1 [==============================] - 0s 28ms/step


array([[-0.38135356],
       [-0.38135356]], dtype=float32)